In [1]:
import pandas as pd
import numpy as np
import pydicom 
import os
import matplotlib
import matplotlib.pyplot as plt

from skimage.measure import label,regionprops
from skimage.segmentation import clear_border

from sklearn.cluster import KMeans
from skimage import morphology
from skimage import measure
from skimage.transform import resize

import scipy

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
submission = pd.read_csv('sample_submission.csv')

In [3]:
patients = train.Patient.unique()

In [4]:
#this is a list of patients for which there is a border around the scans 
patients_with_border = ['ID00419637202311204720264', 'ID00240637202264138860065', 'ID00122637202216437668965', 'ID00086637202203494931510', 
                      'ID00094637202205333947361', 'ID00067637202189903532242', 'ID00014637202177757139317']

In [5]:
def load_scan(path):
    slices = [pydicom.read_file(path + '/' + s) for s in os.listdir(path)]
    slices.sort(key = lambda x: int(x.InstanceNumber))
    return slices

def scan_28(scan):
    
    if len(scan) > 28:
        n = int(round(len(scan)/28))
        smaller = []
        j=0
        while j < len(scan):
            smaller.append(scan[j])
            j += n
        return smaller
    else:
        return scan
    
def load_scan_28(path):
    return scan_28(load_scan(path))

In [6]:
def get_pixels_hu(scans):
    image = np.stack([s.pixel_array for s in scans])
    # Convert to int16 (from sometimes int16), 
    # should be possible as values should always be low enough (<32k)
    image = image.astype(np.int16)

    # Set outside-of-scan pixels to 1
    # The intercept is usually -1024, so air is approximately 0
    image[image == -2000] = 0
    image[image == -3000] = 0
    
    # Convert to Hounsfield units (HU)
    intercept = scans[0].RescaleIntercept
    slope = scans[0].RescaleSlope
    
    if slope != 1:
        image = slope * image.astype(np.float64)
        image = image.astype(np.int16)
        
    image += np.int16(intercept)
    
    return np.array(image, dtype=np.int16)

In [7]:
def crop_center(img):
    y,x = img.shape    
    return img[y//2 - 256:y//2 + 256,x//2 - 256:x//2 + 256]

def shape_512(array):
    
    array_512 = np.zeros([array.shape[0],512,512])
    
    for i in range(array.shape[0]):
        array_512[i] = crop_center(array[i])
        
    return array_512

In [8]:
def scale_img(img):
    return resize(img, (512, 512))

def scale_array(array):
    
    array_512 = np.zeros([array.shape[0],512,512])
    
    for i in range(array.shape[0]):
        array_512[i] = scale_img(array[i])
        
    return array_512

In [9]:
def first_treatment(path):
    pat = path.split('/')[1]
    scan_28 = load_scan_28(path)
    imgs = get_pixels_hu(scan_28)
    if pat in patients_with_border:
        return shape_512(imgs)
    elif imgs[1].shape[0] > 512:
        return scale_array(imgs)
    else:
        return imgs        

In [10]:
def make_lungmask(imgs):

    vert_size = imgs.shape[0]
    row_size= imgs.shape[1]
    col_size = imgs.shape[2]

    mean = np.mean(imgs)
    std = np.std(imgs)
    imgs = imgs-mean
    imgs = imgs/std
    # Find the average pixel value near the lungs
    # to renormalize washed out images
    middle = imgs[int(vert_size/5):int(vert_size/5*4),int(col_size/5):int(col_size/5*4),int(row_size/5):int(row_size/5*4)] 
    mean = np.mean(middle)
    max = np.max(imgs)
    min = np.min(imgs)
    # To improve threshold finding, I'm moving the 
    # underflow and overflow on the pixel spectrum
    imgs[imgs==max]=mean
    imgs[imgs==min]=mean

    #
    # Using Kmeans to separate foreground (soft tissue / bone) and background (lung/air)
    #
    kmeans = KMeans(n_clusters=2).fit(np.reshape(middle,[np.prod(middle.shape),1]))
    centers = sorted(kmeans.cluster_centers_.flatten())
    threshold = np.mean(centers)
    thresh_imgs = np.where(imgs<threshold,1.0,0.0)  # threshold the image
    for i in range(vert_size):
        thresh_imgs[i] = clear_border(thresh_imgs[i])

    eroded = morphology.erosion(thresh_imgs,np.ones([1,4,4]))
    dilation = morphology.dilation(eroded,np.ones([1,8,8]))

    labels = measure.label(dilation) # Different labels are displayed in different colors
    label_vals = np.unique(labels)
    regions = measure.regionprops(labels)
    good_labels = []
    for prop in regions:
        #B = prop.bbox
        if prop.area > 50000 and  120 < prop.centroid[1] < 410 :
            #and B[2]>200 and B[0]<300:
            good_labels.append(prop.label)

    mask = np.ndarray([vert_size,row_size,col_size],dtype=np.int8)
    mask[::] = 0

    for N in good_labels:
        mask = mask + np.where(labels==N,1,0)
    mask = morphology.dilation(mask,np.ones([1,8,8])) # one last dilation

    return mask*(imgs+3)

def best_lung_img(imgs):
    lungs = make_lungmask(imgs)
    img = lungs[1]
    pixel_nr = (img != 0).sum()
    
    for i in range(2,lungs.shape[0]):
        tmp_img = lungs[i]
        tmp_pixel_nr = (tmp_img != 0).sum() 
        if tmp_pixel_nr > pixel_nr:
            img = tmp_img
            pixel_nr = tmp_pixel_nr
            
    return img

def best_lung_slice(path):
    imgs = first_treatment(path)
    return best_lung_img(imgs)

In [11]:
for pat in patients:
    np.savetxt('best_lung_slice/' + pat  + '.txt', best_lung_slice('train/' + pat)) 

In [12]:
def load_best_slices(path):
    ind = []
    for file in os.listdir(path):
        if file.split('.')[1] == 'txt':
            ind.append(file.split('.')[0])
        
    df = pd.DataFrame(index = ind, columns= ['CT'])
    for ind in df.index:
        df.loc[ind].CT = np.loadtxt(path + ind + '.txt')
    
    return df

df = load_best_slices('best_lung_slice/')

import matplotlib

for ind in df.index.to_list():
    matplotlib.image.imsave('lung_images/'+ ind +'.jpeg', df.loc[ind].CT)

In [13]:
# lst = df.CT.apply(lambda x: x.min()).to_list()
# lst.sort(reverse=True)
# lst

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0

In [17]:
# def dif_zero_mean(array):
#     return array[array != 0].mean(), array[array != 0].std()

# lst = df.CT.apply(lambda x: dif_zero_mean(x)).to_list()
# lst.sort(reverse=True)
# lst

[(3.5853151529874956, 0.3781429506987143),
 (3.4566860011103704, 0.34269436384924074),
 (3.3878537883531985, 0.5824864716099972),
 (3.366641135413671, 0.3363408466770521),
 (3.36603746368044, 0.3243364837931565),
 (3.3535830844264467, 0.3280985180500491),
 (3.349532431768563, 0.3169483378370891),
 (3.348762655211393, 0.3268599081902582),
 (3.345303526170793, 0.36197644483163743),
 (3.3375503956808013, 0.779875037736318),
 (3.333238617539019, 0.3003468973878473),
 (3.3277636984684893, 0.3713127122219845),
 (3.3111432970856667, 0.30610204874193614),
 (3.3048889356639783, 0.31536064474621056),
 (3.284015445596549, 0.33939555179005804),
 (3.273081393548148, 0.29508871468618725),
 (3.2690989259518903, 0.48391356844643235),
 (3.2587810133634036, 0.32173410630068167),
 (3.2533504545192238, 0.3545439142931341),
 (3.24661202020694, 0.33693631683769937),
 (3.245128746694513, 0.3141103812665131),
 (3.2408733788345936, 0.4410138612207249),
 (3.233005291411882, 0.31738359173151015),
 (3.21712411513